In [1]:
import numpy as np
import pyradiosky
import astropy.units as units
from astropy.units import Quantity
from astropy.coordinates import (
    Latitude,
    Longitude,
    SkyCoord
)
import matplotlib.pyplot as plt
import json

In [2]:
json_path = "/Users/ruby/Astro/rlb_LWA/LWA_skymodels/bright_sources.json"
save_path = "/Users/ruby/Astro/rlb_LWA/LWA_skymodels/cyg_cas.skyh5"
output_freq_hz = 56.0e6

In [3]:
bright_sources_file = open(json_path)
data = json.load(bright_sources_file)
bright_sources_file.close()

In [4]:
data[0]['components']

[{'name': '1',
  'ra': '19h59m29.990s',
  'dec': '+40d43m57.53s',
  'I': 43170.55527073293,
  'freq': 1000000.0,
  'index': [0.085, -0.178],
  'major-fwhm': 127.86780196141683,
  'minor-fwhm': 22.459884076169928,
  'position-angle': -74.50271323639498},
 {'name': '2',
  'ra': '19h59m24.316s',
  'dec': '+40d44m50.70s',
  'I': 6374.4647292670625,
  'freq': 1000000.0,
  'index': [0.085, -0.178],
  'major-fwhm': 183.42701763410113,
  'minor-fwhm': 141.44188315233822,
  'position-angle': 43.449049376516}]

In [5]:
data[0]

{'ref': 'Michael (2016-07-30T10:52:45)',
 'name': 'Cyg A',
 'components': [{'name': '1',
   'ra': '19h59m29.990s',
   'dec': '+40d43m57.53s',
   'I': 43170.55527073293,
   'freq': 1000000.0,
   'index': [0.085, -0.178],
   'major-fwhm': 127.86780196141683,
   'minor-fwhm': 22.459884076169928,
   'position-angle': -74.50271323639498},
  {'name': '2',
   'ra': '19h59m24.316s',
   'dec': '+40d44m50.70s',
   'I': 6374.4647292670625,
   'freq': 1000000.0,
   'index': [0.085, -0.178],
   'major-fwhm': 183.42701763410113,
   'minor-fwhm': 141.44188315233822,
   'position-angle': 43.449049376516}]}

In [6]:
data[0]

{'ref': 'Michael (2016-07-30T10:52:45)',
 'name': 'Cyg A',
 'components': [{'name': '1',
   'ra': '19h59m29.990s',
   'dec': '+40d43m57.53s',
   'I': 43170.55527073293,
   'freq': 1000000.0,
   'index': [0.085, -0.178],
   'major-fwhm': 127.86780196141683,
   'minor-fwhm': 22.459884076169928,
   'position-angle': -74.50271323639498},
  {'name': '2',
   'ra': '19h59m24.316s',
   'dec': '+40d44m50.70s',
   'I': 6374.4647292670625,
   'freq': 1000000.0,
   'index': [0.085, -0.178],
   'major-fwhm': 183.42701763410113,
   'minor-fwhm': 141.44188315233822,
   'position-angle': 43.449049376516}]}

In [7]:
data[1]

{'name': 'Cas A',
 'components': [{'Q': 0.0,
   'minor-fwhm': 84.1,
   'V': 0.0,
   'major-fwhm': 208.89999999999998,
   'name': '1',
   'ra': '23h23m12.780s',
   'freq': 1000000.0,
   'index': [-0.77],
   'I': 205291.01635813876,
   'dec': '+58d50m41.00s',
   'U': 0.0,
   'position-angle': 38.9},
  {'Q': 0.0,
   'minor-fwhm': 121.9,
   'V': 0.0,
   'major-fwhm': 230.89999999999998,
   'name': '2',
   'ra': '23h23m28.090s',
   'freq': 1000000.0,
   'index': [-0.77],
   'I': 191558.43164385832,
   'dec': '+58d49m18.10s',
   'U': 0.0,
   'position-angle': 43.8},
  {'Q': 0.0,
   'minor-fwhm': 63.4649,
   'V': 0.0,
   'major-fwhm': 173.26,
   'name': '3',
   'ra': '23h23m20.880s',
   'freq': 1000000.0,
   'index': [-0.77],
   'I': 159054.81199800296,
   'dec': '+58d50m49.92s',
   'U': 0.0,
   'position-angle': 121.902}]}

In [8]:
len(data[1]["components"])

3

In [9]:
len(data)

2

In [10]:
def interpolate_flux(
    start_flux, start_freq_mhz, end_freq_mhz,
    spec_index_1=0, spec_index_2=0
):
    end_flux = start_flux*(end_freq_mhz/start_freq_mhz)**(
        spec_index_1 + spec_index_2*np.log10(end_freq_mhz/start_freq_mhz)
    )
    return end_flux

In [11]:
Ncomponents = sum([len(source['components']) for source in data])
Nfreqs = 1


In [12]:
ras = Longitude(np.zeros(Ncomponents, dtype=float), units.deg)
decs = Latitude(np.zeros(Ncomponents, dtype=float), units.deg)
stokes = Quantity(np.zeros((4, Nfreqs, Ncomponents), dtype=float), 'Jy')
spec_indexes = np.zeros(Ncomponents, dtype=float)
names = np.empty(Ncomponents, dtype=str)
extended_groups = np.zeros(Ncomponents, dtype=object)
reference_frequency = Quantity(np.full(Ncomponents, output_freq_hz), "hertz")

In [13]:
comp_ind = 0
for source in data:
    for comp in source["components"]:
        comp_coord = SkyCoord(comp['ra'], comp['dec'], frame='icrs')
        ras[comp_ind] = comp_coord.ra
        decs[comp_ind] = comp_coord.dec
        spec_index_1 = comp['index'][0]
        if len(comp['index']) > 1:
            spec_index_2 = comp['index'][1]
        else:
            spec_index_2 = 0.
        flux_I = interpolate_flux(
            comp['I'], comp['freq'], output_freq_hz, 
            spec_index_1=spec_index_1, spec_index_2=spec_index_2
        )
        stokes[0, 0, comp_ind] = flux_I * units.Jy
        spec_indexes[comp_ind] = spec_index_1
        names[comp_ind] = '{}_{}'.format(source['name'], comp['name'])
        extended_groups[comp_ind] = source['name']
        comp_ind += 1

In [14]:
catalog = pyradiosky.SkyModel(
    name=names,
    ra=ras,
    dec=decs,
    stokes=stokes,
    spectral_type="spectral_index",
    reference_frequency=reference_frequency,
    spectral_index=spec_indexes,
    extended_model_group = extended_groups,
)
catalog.check()

True

In [15]:
catalog.write_skyh5(save_path)